In [2]:
import os

import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition, sklearn.svm
import librosa
import librosa.display

import utils
plt.rcParams['figure.figsize'] = (13, 5)

from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.metrics import confusion_matrix
import itertools
from sklearn.model_selection import RandomizedSearchCV
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from collections import Counter
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from scipy import interp
from itertools import cycle
from scikitplot.metrics import plot_roc
from scikitplot.metrics import plot_precision_recall
from scikitplot.metrics import plot_cumulative_gain
from scikitplot.metrics import plot_lift_curve


from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import CondensedNearestNeighbour
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier

In [3]:
features = pd.read_csv('data/fma_metadata/df_senzaOutliers.csv',index_col=[0], header=[0,1,2])

features

feature    chroma_stft       mfcc                                    \
statistics        mean        max                                     
number              01         01         03          04         07   
track_id                                                              
2             0.696063  28.657707  52.424423  103.289833  37.843021   
3             0.449602  29.380682  76.742714  137.245010  55.662521   
5             0.481160 -40.500740  50.373932  112.312531  29.266182   
10            0.388109  20.196968  60.412815   78.466721  21.597277   
134           0.438062 -41.723225  58.687328  125.922821  37.450359   
...                ...        ...        ...         ...        ...   
155297        0.408626   3.826046  94.208336   48.085751  26.723190   
155298        0.439034 -72.790154  81.013275  124.569000  29.571426   
155306        0.369401  -7.814734  88.024063  116.180260  29.388819   
155307        0.357939 -13.889938  62.219879   93.028778  41.311356   
155314        0.551894 -15.305521  32.737503   95.054657  25.620636   

feature                                                                ...  \
statistics                   mean           min                        ...   
number             11          02            02         04         05  ...   
track_id                                                               ...   
2           27.737400  116.696678 -1.421085e-14 -51.569004 -97.879890  ...   
3           26.740063  120.158501 -1.851665e+01 -12.548506 -87.048355  ...   
5           27.197964  132.215073 -6.228398e+01 -24.319845 -74.060310  ...   
10          24.780317  157.040085  5.151435e-14 -50.578659 -45.113949  ...   
134         36.486378  125.521309 -9.427860e-01 -90.334747 -67.671829  ...   
...               ...         ...           ...        ...        ...  ...   
155297      19.866936  163.954620 -1.776357e-14 -29.221125 -28.104187  ...   
155298      38.472843  160.770218 -7.993606e-14  -7.314378 -69.236115  ...   
155306      47.185654  177.614151  5.329071e-15 -49.451691 -70.841232  ...   
155307      33.690544  124.228142 -5.892177e+00 -50.547176 -99.119080  ...   
155314      34.402496  166.692734 -2.464827e-01 -55.172737 -55.477066  ...   

feature                          spectral_bandwidth spectral_contrast  \
statistics        std                      kurtosis               max   
number             02         04                 01                03   
track_id                                                                
2           38.569584  20.767921           3.874384         39.333603   
3           41.191982  22.025253           2.383340         42.141411   
5           39.367599  24.319347           0.895442         39.017235   
10          44.409843  14.013119           0.437628         45.410881   
134         47.266518  36.841534           0.372903         41.526760   
...               ...        ...                ...               ...   
155297      30.287067  12.060067          20.706787         40.720093   
155298      39.359367  18.604885           5.868375         39.627396   
155306      40.537926  28.671246           4.375974         40.628235   
155307      35.382370  22.123117          -0.383395         48.715477   
155314      26.786989  22.369673           0.011010         39.460281   

feature                           tonnetz metadata                              
statistics                  min       std      set         track                
number             04        02        01   subset     genre_top genre_top_Cat  
track_id                                                                        
2           31.513493  2.296826  0.019809    small       Hip-Hop             7  
3           31.473713  0.652681  0.026316   medium       Hip-Hop             7  
5           33.707951  2.314010  0.025550    small       Hip-Hop             7  
10          28.711344  4.415900  0.021413    small           Pop            12  
134         32.2243

In [4]:
attributes = [col for col in features.columns if (col != ('metadata','track','genre_top') and 
                                                  col != ('metadata','set','subset')  and
                                                 col != ('metadata','track','genre_top_Cat'))] 

X = features[attributes].values
y = features[('metadata','track','genre_top')]

df_SOLO_FEATURES=features.drop([('metadata','track','genre_top'),('metadata','set','subset'),('metadata','track','genre_top_Cat')], axis = 1)
df_SOLO_FEATURES.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24752 entries, 2 to 155314
Data columns (total 20 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   (chroma_stft, mean, 01)             24752 non-null  float64
 1   (mfcc, max, 01)                     24752 non-null  float64
 2   (mfcc, max, 03)                     24752 non-null  float64
 3   (mfcc, max, 04)                     24752 non-null  float64
 4   (mfcc, max, 07)                     24752 non-null  float64
 5   (mfcc, max, 11)                     24752 non-null  float64
 6   (mfcc, mean, 02)                    24752 non-null  float64
 7   (mfcc, min, 02)                     24752 non-null  float64
 8   (mfcc, min, 04)                     24752 non-null  float64
 9   (mfcc, min, 05)                     24752 non-null  float64
 10  (mfcc, min, 11)                     24752 non-null  float64
 11  (mfcc, skew, 04)                    2475

In [5]:
scaler = StandardScaler()
scaler.fit(X)

X = scaler.transform(X)
X

array([[ 1.87392878,  0.61578261, -0.82184851, ..., -0.92738463,
        -0.63724001, -1.33922307],
       [-0.10619751,  0.62498146, -0.01371958, ..., -0.93542456,
        -1.96365133, -0.46870945],
       [ 0.14734683, -0.26416207, -0.88998905, ..., -0.48385893,
        -0.62337644, -0.57117433],
       ...,
       [-0.75055752,  0.15172173,  0.36117456, ..., -0.16252053,
        -0.30404413,  1.3492635 ],
       [-0.84264233,  0.07442323, -0.49633257, ...,  0.020341  ,
        -0.49616715,  0.72700404],
       [ 0.71564291,  0.05641192, -1.47607088, ..., -0.91371874,
        -0.20512381, -0.32985258]])

In [6]:
df_features = pd.DataFrame(X,columns=df_SOLO_FEATURES.columns, index=df_SOLO_FEATURES.index)
df_features[('metadata','track','genre_top')]=y
df_features.head()

feature    chroma_stft      mfcc                                          \
statistics        mean       max                                           
number              01        01        03        04        07        11   
track_id                                                                   
2             1.873929  0.615783 -0.821849  0.210778 -0.093897 -0.480419   
3            -0.106198  0.624981 -0.013720  1.454959  1.226926 -0.579583   
5             0.147347 -0.264162 -0.889989  0.541386 -0.729633 -0.534054   
10           -0.600249  0.508132 -0.556384 -0.698787 -1.298071 -0.774439   
134          -0.198917 -0.279716 -0.613724  1.040093 -0.123002  0.389485   

feature                                             ...                      \
statistics      mean       min                      ...      skew       std   
number            02        02        04        05  ...        04        01   
track_id                                            ...                       
2          -0.925824  0.472560 -0.318376 -2.148282  ...  0.450247  0.630834   
3          -0.815355 -0.169676  1.288356 -1.578578  ...  0.337926  1.226592   
5          -0.430625 -1.687714  0.803652 -0.895448  ...  0.207039  0.512383   
10          0.361551  0.472560 -0.277597  0.627037  ... -0.518258  0.842674   
134        -0.644226  0.439860 -1.914619 -0.559435  ... -0.386534  0.844228   

feature                        spectral_bandwidth spectral_contrast            \
statistics                               kurtosis               max             
number            02        04                 01                03        04   
track_id                                                                        
2           0.136186  0.155440          -0.125569         -0.723865 -0.927385   
3           0.389213  0.369593          -0.199605         -0.142222 -0.935425   
5           0.213184  0.760329          -0.273484         -0.789402 -0.483859   
10          0.699694 -0.995056          -0.296217          0.535056 -1.493732   
134         0.975325  2.893142          -0.299430         -0.269548 -0.783718   

feature                tonnetz  metadata  
statistics       min       std     track  
number            02        01 genre_top  
track_id                                  
2          -0.637240 -1.339223   Hip-Hop  
3          -1.963651 -0.468709   Hip-Hop  
5          -0.623376 -0.571174   Hip-Hop  
10          1.072320 -1.124712       Pop  
134        -1.465178 -0.744009   Hip-Hop  

[5 rows x 21 columns]

In [8]:
attributes = [col for col in features.columns if (col != ('metadata','track','genre_top') and 
                                                  col != ('metadata','set','subset')  and
                                                 col != ('metadata','track','genre_top_Cat'))] 

X = features[attributes].values
y = features[('metadata','track','genre_top')]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1, stratify=y_train)

In [16]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [7]:
DTREE=DecisionTreeClassifier(criterion='gini', max_depth=14, min_samples_split=30, min_samples_leaf=20,class_weight="balanced")
SVM=SVC(C=10,gamma=0.1,kernel='rbf',probability=True,class_weight='balanced')

In [29]:
param_grid = {'base_estimator': [DTREE,SVM]}


clf1 = GridSearchCV(AdaBoostClassifier(), 
                         param_grid, cv=StratifiedKFold(5), scoring='f1_macro')

clf1 = clf1.fit(X_train, y_train)
report(clf1.cv_results_, n_top=5)

KeyboardInterrupt: 

In [9]:
DTREE=DecisionTreeClassifier(criterion='gini', max_depth=14, min_samples_split=50, min_samples_leaf=50)

clf1 = AdaBoostClassifier(base_estimator=DTREE, n_estimators=500, random_state=0)
clf1 = clf1.fit(X_train, y_train)


In [10]:
y_pred_tr = clf1.predict(X_train)
print('Train Accuracy %s' % accuracy_score(y_train, y_pred_tr))
print('Train F1-score %s' % f1_score(y_train, y_pred_tr, average='macro'))
print('Val Recall %s' % recall_score(y_train, y_pred_tr,average='macro'))
print('Val Precision %s' % precision_score(y_train, y_pred_tr,average='macro'))

print(classification_report(y_train, y_pred_tr))

confusion_matrix(y_train, y_pred_tr)


Train Accuracy 0.8808080808080808
Train F1-score 0.9216522258629987
Val Recall 0.9033151228667662
Val Precision 0.9443033642194841
                     precision    recall  f1-score   support

              Blues       1.00      1.00      1.00        44
          Classical       0.87      0.93      0.90       371
            Country       1.00      0.97      0.99       107
     Easy Listening       1.00      1.00      1.00        13
         Electronic       0.83      0.89      0.86      3742
       Experimental       0.87      0.76      0.81      1291
               Folk       0.97      0.93      0.95       910
            Hip-Hop       0.83      0.80      0.82      1318
       Instrumental       0.87      0.81      0.84       793
      International       0.99      0.85      0.91       607
               Jazz       1.00      0.93      0.96       230
Old-Time / Historic       1.00      0.99      1.00       302
                Pop       1.00      0.76      0.86       709
              

array([[  44,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [   0,  346,    0,    0,    1,    5,    0,    0,   19,    0,    0,
           0,    0,    0,    0,    0],
       [   0,    0,  104,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    3,    0,    0],
       [   0,    0,    0,   13,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0],
       [   0,    7,    0,    0, 3326,   53,    2,  150,   31,    2,    0,
           0,    1,  170,    0,    0],
       [   0,   13,    0,    0,  140,  986,    2,   11,   24,    0,    0,
           0,    0,  115,    0,    0],
       [   0,    2,    0,    0,    4,    5,  844,    1,    4,    1,    0,
           0,    0,   49,    0,    0],
       [   0,    0,    0,    0,  208,    3,    0, 1056,    0,    0,    0,
           0,    0,   51,    0,    0],
       [   0,   24,    0,    0,   36,   38,    1,    0,  646,    0,    0,
           0,    0,   

In [48]:
y_pred_tr = clf1.predict(X_val)
print('Train Accuracy %s' % accuracy_score(y_val, y_pred_tr))
print('Train F1-score %s' % f1_score(y_val, y_pred_tr, average='macro'))
print('Val Recall %s' % recall_score(y_val, y_pred_tr,average='macro'))
print('Val Precision %s' % precision_score(y_val, y_pred_tr,average='macro'))

print(classification_report(y_val, y_pred_tr))

confusion_matrix(y_val, y_pred_tr)


Train Accuracy 0.546960210058574
Train F1-score 0.3031431600198858
Val Recall 0.28491475722950865
Val Precision 0.437550025292871
                     precision    recall  f1-score   support

              Blues       0.00      0.00      0.00        15
          Classical       0.59      0.49      0.53       123
            Country       0.00      0.00      0.00        35
     Easy Listening       0.00      0.00      0.00         4
         Electronic       0.56      0.72      0.63      1248
       Experimental       0.34      0.26      0.30       431
               Folk       0.55      0.46      0.50       303
            Hip-Hop       0.50      0.42      0.46       440
       Instrumental       0.38      0.24      0.29       265
      International       0.67      0.19      0.30       203
               Jazz       1.00      0.01      0.03        76
Old-Time / Historic       0.99      0.83      0.90       100
                Pop       0.09      0.01      0.02       237
               

C:\Users\Federico\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Federico\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[   0,    0,    0,    0,    5,    1,    3,    1,    0,    0,    0,
           0,    1,    4,    0,    0],
       [   0,   60,    0,    0,    6,   20,    8,    0,   19,    0,    0,
           0,    0,   10,    0,    0],
       [   0,    0,    0,    0,    4,    0,    2,    0,    0,    1,    0,
           0,    0,   28,    0,    0],
       [   0,    0,    0,    0,    1,    0,    0,    0,    2,    0,    0,
           0,    0,    1,    0,    0],
       [   0,    4,    0,    0,  894,   42,    6,   98,   22,    0,    0,
           0,    3,  179,    0,    0],
       [   0,    7,    0,    0,  144,  113,   13,   12,   19,    3,    0,
           0,    2,  118,    0,    0],
       [   0,    1,    0,    0,   23,   16,  139,    2,   10,    5,    0,
           0,    4,  103,    0,    0],
       [   0,    0,    0,    0,  139,   12,    2,  185,    1,    1,    0,
           0,    7,   93,    0,    0],
       [   0,   22,    0,    0,   60,   33,   19,    4,   64,    0,    0,
           1,    2,   

In [45]:
y_pred = clf1.predict(X_test)

print('Val Accuracy %0.4f' % accuracy_score(y_test, y_pred))


print('Train F1-score %s' % f1_score(y_test, y_pred, average='macro'))
print('Val Recall %s' % recall_score(y_test, y_pred,average='macro'))
print('Val Precision %s' % precision_score(y_test, y_pred,average='macro'))

print(classification_report(y_test, y_pred))

confusion_matrix(y_test, y_pred)

Val Accuracy 0.5292
Train F1-score 0.2850033156041076
Val Recall 0.26924999498391466
Val Precision 0.3848165277987734
                     precision    recall  f1-score   support

              Blues       0.00      0.00      0.00        15
          Classical       0.62      0.51      0.56       124
            Country       0.00      0.00      0.00        36
     Easy Listening       0.00      0.00      0.00         4
         Electronic       0.53      0.66      0.59      1248
       Experimental       0.30      0.26      0.28       431
               Folk       0.53      0.39      0.45       303
            Hip-Hop       0.48      0.41      0.45       439
       Instrumental       0.38      0.26      0.31       265
      International       0.51      0.15      0.23       203
               Jazz       0.00      0.00      0.00        76
Old-Time / Historic       0.99      0.78      0.87       100
                Pop       0.23      0.04      0.07       236
               Rock       0

C:\Users\Federico\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Federico\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[   0,    0,    0,    0,    4,    3,    0,    1,    1,    0,    0,
           0,    0,    6,    0,    0],
       [   0,   63,    0,    0,   10,   19,    7,    0,   17,    1,    0,
           0,    0,    7,    0,    0],
       [   0,    0,    0,    0,    2,    2,    4,    1,    0,    0,    0,
           0,    0,   27,    0,    0],
       [   0,    0,    0,    0,    2,    0,    0,    1,    1,    0,    0,
           0,    0,    0,    0,    0],
       [   0,    6,    0,    0,  820,   71,    6,  117,   26,    9,    0,
           0,    4,  189,    0,    0],
       [   0,    6,    0,    0,  150,  112,   14,   13,   21,    3,    0,
           1,    4,  107,    0,    0],
       [   0,    1,    0,    0,   25,   19,  119,    2,   10,    3,    0,
           0,    4,  120,    0,    0],
       [   0,    0,    0,    0,  158,    5,    0,  182,    0,    2,    0,
           0,    7,   85,    0,    0],
       [   0,   17,    0,    0,   68,   42,   13,    1,   69,    2,    0,
           0,    0,   